Restarted aspire (Python 3.8.18)

In [11]:
import aspire
import numpy as np
import pandas as pd
from aspire.operators import RadialCTFFilter
from aspire.source.simulation import Simulation
from aspire.volume import LegacyVolume, Volume
from utils import volsCovarEigenvec
import time
from covar_estimation import im_stack_backward
import torch
# Specify parameters
img_size = 15  # image size in square
num_imgs = 2000  # number of images
dtype = np.float32


rank = 1
c = rank + 1
vols = LegacyVolume(
    L=img_size,
    C=c,
    dtype=dtype,
).generate()
vols -= np.mean(vols,axis=0)
sim = Simulation(
    #unique_filters=[RadialCTFFilter(defocus=d) for d in np.linspace(1.5e4, 2.5e4, 7)],
    n=num_imgs,
    vols=vols,
    dtype=dtype,
    amplitudes=1,
    offsets = 0
)

vectorsGD = torch.tensor(volsCovarEigenvec(vols).asnumpy(),requires_grad = False)

2024-03-22 18:40:30,347 INFO [aspire.source.image] Creating Simulation with 2000 images.


In [12]:
%load_ext autoreload
%autoreload 2

from covar_sgd import CovarDataset,Covar,CovarTrainer,dataset_collate
cds = CovarDataset(sim,rank = rank)
covar = Covar(resolution=img_size,rank=rank,norm_factor=cds.im_norm_factor)
batch_size = 20
learning_rate = 1e-5
momentum = 0.9
device = torch.device('cuda:0')
dataloader = torch.utils.data.DataLoader(cds,batch_size = batch_size,shuffle = False,collate_fn=dataset_collate)
optimizer = torch.optim.SGD(covar.parameters(),lr = learning_rate,momentum = momentum)
trainer = CovarTrainer(covar,dataloader,optimizer,device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
trainer.train(1)

tensor(9.7106, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(13.2256, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(11.0746, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(10.3704, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(9.9364, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(11.3395, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(11.9322, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(11.4423, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(12.8222, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(10.2688, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(13.6417, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(11.1644, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(13.2421, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(12.8006, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(15.4705, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(11.2478, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(9.2102, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(11.7175, d

In [18]:
images,plans = cds[:]
images = images.to(device)
#covar = Covar(resolution=img_size,rank=rank,norm_factor=cds.im_norm_factor)
#covar = covar.to(device)
#a = covar.cost(images[0],plans[0])
covarGD = Covar(resolution=img_size,rank=rank,norm_factor=cds.im_norm_factor,vectors=vectorsGD)
covarGD = covarGD.to(device)
b = covarGD.cost(images,plans)

print(a,b)

tensor(0.0579, device='cuda:0', grad_fn=<MeanBackward1>) tensor(6.9141e-08, device='cuda:0', grad_fn=<MeanBackward1>)
